In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/val/val_week1_2.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/test/test_week2.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/clean_data_week2.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_2.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_3.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_4.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_1.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_5.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/val/val_week1_2_3.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/test/test_week3.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/train/clean_data_week3.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw

In [2]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import StratifiedKFold
import time
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support

/tmp/ipykernel_23/1167260223.py:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [3]:
five_fold_files = {
    "week1": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/train/5-folds/data_part_5.csv"
    ],
    "week2": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/train/5-folds/data_part_5.csv"
    ],
    "week3": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/train/5-folds/data_part_5.csv"
    ],
    "week4": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/train/5-folds/data_part_5.csv"
    ]
}

file_validataion = {
    "week1": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/val/val_week1.csv"
    ],
    "week2": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/val/val_week1_2.csv"
    ],
    "week3": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/val/val_week1_2_3.csv"
    ],
    "week4": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/val/val_week1_2_3_4.csv"
    ]
}

file_test = {
    "week1": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/test/test_week1.csv"
    ],
    "week2": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/test/test_week2.csv"
    ],
    "week3": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/test/test_week3.csv"
    ],
    "week4": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/test/test_week4.csv"
    ]
}

In [4]:
# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    return focal_loss_fixed

# Tạo hàm train cho từng tuần
def train_week_model(week_number, file_paths_train, file_validataion):
   # Đọc dữ liệu
    train_data = pd.read_csv(file_paths_train)
    val_data = pd.read_csv(file_validataion)
    
    # Tách đặc trưng và nhãn
    X_train = train_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_train = train_data["classification_encoded"]

    X_val = val_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_val = val_data["classification_encoded"]
    

    # Áp dụng Over-sampling cho dữ liệu huấn luyện bằng SMOTE
    oversampler = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_res, y_train_res = oversampler.fit_resample(X_train, y_train)

    # One-hot encode nhãn
    y_train_res = tf.keras.utils.to_categorical(y_train_res, num_classes=5)
    y_val = tf.keras.utils.to_categorical(y_val, num_classes=5)

    # Xây dựng mô hình 4-layer stacked LSTM
    def build_model(hp):
        model = tf.keras.Sequential()
        
        # LSTM layer 1
        model.add(layers.LSTM(
            units=hp.Int('units_1', min_value=32, max_value=256, step=32),
            return_sequences=True,
            input_shape=(X_train_res.shape[1], 1)
        ))
        model.add(layers.Dropout(rate=hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))
        
        # LSTM layer 2
        model.add(layers.LSTM(
            units=hp.Int('units_2', min_value=32, max_value=256, step=32),
            return_sequences=True
        ))
        model.add(layers.Dropout(rate=hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)))
        
        # LSTM layer 3
        model.add(layers.LSTM(
            units=hp.Int('units_3', min_value=32, max_value=256, step=32),
            return_sequences=True
        ))
        model.add(layers.Dropout(rate=hp.Float('dropout_3', min_value=0.1, max_value=0.5, step=0.1)))
        
        # LSTM layer 4
        model.add(layers.LSTM(
            units=hp.Int('units_4', min_value=32, max_value=256, step=32),
            return_sequences=False
        ))
        model.add(layers.Dropout(rate=hp.Float('dropout_4', min_value=0.1, max_value=0.5, step=0.1)))

        # Lớp đầu ra
        model.add(layers.Dense(5, activation='softmax'))

        # Compile với Focal Loss
        model.compile(optimizer=tf.keras.optimizers.Adam(
                          learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                      loss=focal_loss(gamma=2., alpha=0.25),
                      metrics=['accuracy'])
        return model

    # Khởi tạo RandomSearch tuner
    tuner = RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=5,
        executions_per_trial=1,
        directory='my_dir',
        project_name=f'lstm_tuning_week{week_number}'
    )

    # Tìm kiếm siêu tham số tốt nhất
    tuner.search(X_train_res, y_train_res,
                 epochs=15,
                 validation_data=(X_val, y_val),
                 batch_size=32)

    # Trả về kết quả tối ưu cho tuần
    best_params = tuner.get_best_hyperparameters(num_trials=1)[0]
    return best_params


In [5]:
# Định nghĩa đường dẫn đến dữ liệu cho từng tuần
file_paths_train = {
    "week1": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/train/clean_data_week1.csv",
    "week2": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/train/clean_data_week2.csv",
    "week3": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/train/clean_data_week3.csv",
    "week4": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/train/clean_data_week4.csv"
}

file_validataion = {
    "week1": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/val/val_week1.csv",
    "week2": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/val/val_week1_2.csv",
    "week3": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/val/val_week1_2_3.csv",
    "week4": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/val/val_week1_2_3_4.csv"
}

In [6]:
# Tìm tham số tốt nhất cho từng tuần
best_params_week1 = train_week_model(1, file_paths_train["week1"], file_validataion["week1"])
best_params_week2 = train_week_model(2, file_paths_train["week2"], file_validataion["week2"])
best_params_week3 = train_week_model(3, file_paths_train["week3"], file_validataion["week3"])
best_params_week4 = train_week_model(4, file_paths_train["week4"], file_validataion["week4"])

# In thông tin chi tiết các tham số tối ưu
print("Best Parameters for Week 1:")
for param_name in best_params_week1.values.keys():
    print(f"{param_name}: {best_params_week1.get(param_name)}")

print("\nBest Parameters for Week 2:")
for param_name in best_params_week2.values.keys():
    print(f"{param_name}: {best_params_week2.get(param_name)}")

print("\nBest Parameters for Week 3:")
for param_name in best_params_week3.values.keys():
    print(f"{param_name}: {best_params_week3.get(param_name)}")

print("\nBest Parameters for Week 4:")
for param_name in best_params_week4.values.keys():
    print(f"{param_name}: {best_params_week4.get(param_name)}")


Trial 5 Complete [00h 03m 42s]
val_accuracy: 0.9987797141075134

Best val_accuracy So Far: 0.9987797141075134
Total elapsed time: 00h 17m 53s
Best Parameters for Week 1:
units_1: 64
dropout_1: 0.1
units_2: 128
dropout_2: 0.1
units_3: 192
dropout_3: 0.2
units_4: 64
dropout_4: 0.2
learning_rate: 0.004531589379324198

Best Parameters for Week 2:
units_1: 160
dropout_1: 0.1
units_2: 128
dropout_2: 0.4
units_3: 192
dropout_3: 0.30000000000000004
units_4: 160
dropout_4: 0.5
learning_rate: 0.001800627430497124

Best Parameters for Week 3:
units_1: 32
dropout_1: 0.30000000000000004
units_2: 256
dropout_2: 0.4
units_3: 128
dropout_3: 0.4
units_4: 64
dropout_4: 0.5
learning_rate: 0.002577673485035491

Best Parameters for Week 4:
units_1: 160
dropout_1: 0.30000000000000004
units_2: 128
dropout_2: 0.5
units_3: 64
dropout_3: 0.30000000000000004
units_4: 160
dropout_4: 0.30000000000000004
learning_rate: 0.0013346639042796605


In [7]:
# Danh sách tham số tốt nhất
best_params = {
    "week1": best_params_week1,
    "week2": best_params_week2,
    "week3": best_params_week3,
    "week4": best_params_week4
}

In [8]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

# Xây dựng mô hình
def build_lstm_model(params):
    model = tf.keras.Sequential()
    
    # LSTM layer 1
    model.add(layers.LSTM(
        units=params.get('units_1', 32),
        return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]))
    )
    model.add(layers.Dropout(rate=params.get('dropout_1', 0.2)))

    # LSTM layer 2
    model.add(layers.LSTM(
        units=params.get('units_2', 32),
        return_sequences=True
    ))
    model.add(layers.Dropout(rate=params.get('dropout_2', 0.2)))

    # LSTM layer 3
    model.add(layers.LSTM(
        units=params.get('units_3', 32),
        return_sequences=True
    ))
    model.add(layers.Dropout(rate=params.get('dropout_3', 0.2)))

    # LSTM layer 4
    model.add(layers.LSTM(
        units=params.get('units_4', 32),
        return_sequences=False
    ))
    model.add(layers.Dropout(rate=params.get('dropout_4', 0.2)))

    # Output layer
    model.add(layers.Dense(5, activation='softmax'))  # 5 classes in output

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
        loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
        metrics=['accuracy']
    )
    
    return model


In [9]:
# Biến lưu kết quả tổng quát
overall_results_5folds = []

# Lặp qua từng tuần
for week, file_paths in five_fold_files.items():
    print(f"\nProcessing {week} with best parameters...")
    params = best_params[week].values
    print(f"best parameters for {week}: {params}")
    
    # Biến lưu kết quả cho từng tuần
    week_results = {
        "week": week,
        "accuracy_per_fold": [],
        "precision_per_label": [],
        "recall_per_label": [],
        "f1_score_per_label": [],
        "confusion_matrices": [],
        "train_times": [],
        "test_times": []
    }

    # Lặp qua từng fold
    for i in range(len(file_paths)):
        print(f"Fold {i+1}: Using file {file_paths[i]} as test set")
        
        # Tải dữ liệu
        test_data = pd.read_csv(file_paths[i])
        train_data = pd.concat([pd.read_csv(file_paths[j]) for j in range(len(file_paths)) if j != i])
        
        # Tách X và y
        X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                           "course_id", "school", "enroll_time", "classification"])
        y_train = to_categorical(train_data['classification_encoded'], num_classes=5)
        
        X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                         "course_id", "school", "enroll_time", "classification"])
        y_test = to_categorical(test_data['classification_encoded'], num_classes=5)

        # Reshape dữ liệu cho LSTM
        X_train = X_train.to_numpy().reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

        # Xây dựng mô hình với tham số tốt nhất
        model = build_lstm_model(params)
        
        # Bắt đầu tính thời gian huấn luyện
        start_train = time.time()
        model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)
        end_train = time.time()
        
        # Bắt đầu tính thời gian kiểm thử
        start_test = time.time()
        y_pred = model.predict(X_test)
        end_test = time.time()
        
        # Tính thời gian và lưu lại
        train_time = end_train - start_train
        test_time = end_test - start_test
        week_results["train_times"].append(train_time)
        week_results["test_times"].append(test_time)

        # Đánh giá mô hình trên tập kiểm thử của fold hiện tại
        _, accuracy = model.evaluate(X_test, y_test, verbose=0)
        week_results["accuracy_per_fold"].append(accuracy)
        
        # Dự đoán
        y_pred_classes = y_pred.argmax(axis=1)
        y_test_classes = y_test.argmax(axis=1)
        
        # Tính các chỉ số cho mỗi fold
        precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
        conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
        
        week_results["precision_per_label"].append(precision)
        week_results["recall_per_label"].append(recall)
        week_results["f1_score_per_label"].append(f1)
        week_results["confusion_matrices"].append(conf_matrix)

    # Tính trung bình cho từng nhãn
    average_precision_per_label = np.mean(week_results["precision_per_label"], axis=0)
    average_recall_per_label = np.mean(week_results["recall_per_label"], axis=0)
    average_f1_per_label = np.mean(week_results["f1_score_per_label"], axis=0)
    average_confusion_matrix = np.mean(week_results["confusion_matrices"], axis=0)
    average_train_time = sum(week_results["train_times"]) / len(week_results["train_times"])
    average_test_time = sum(week_results["test_times"]) / len(week_results["test_times"])
    
    # Tạo DataFrame cho precision, recall, f1-score
    labels = np.unique(y_test_classes)  # Lấy nhãn từ y_test_classes
    metrics_df = pd.DataFrame({
        "Label": labels,
        "Average Precision": average_precision_per_label,
        "Average Recall": average_recall_per_label,
        "Average F1-Score": average_f1_per_label
    })
    
    # Tạo DataFrame cho confusion matrix
    confusion_df = pd.DataFrame(average_confusion_matrix, index=labels, columns=labels)
    
    # In kết quả
    print("\n=== Average Precision, Recall, F1-Score per Label ===")
    print(metrics_df)
    print("\n=== Average Confusion Matrix ===")
    print(confusion_df)

    # Lưu kết quả tuần vào kết quả tổng quát
    week_results["average_train_times"] = average_train_time
    week_results["average_test_times"] = average_test_time
    week_results["average_metrics_df"] = metrics_df
    week_results["average_confusion_matrix"] = confusion_df
    overall_results_5folds.append(week_results)


Processing week1 with best parameters...
best parameters for week1: {'units_1': 64, 'dropout_1': 0.1, 'units_2': 128, 'dropout_2': 0.1, 'units_3': 192, 'dropout_3': 0.2, 'units_4': 64, 'dropout_4': 0.2, 'learning_rate': 0.004531589379324198}
Fold 1: Using file /kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/train/5-folds/data_part_1.csv as test set
Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.6166 - loss: 0.1456 - val_accuracy: 0.6759 - val_loss: 0.1170
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6471 - loss: 0.1237 - val_accuracy: 0.7038 - val_loss: 0.1123
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6572 - loss: 0.1233 - val_accuracy: 0.6786 - val_loss: 0.1266
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.6655 - loss: 0.1198 - val_accuracy: 0.7183 - val_loss: 0.1065
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6621 - loss: 0.1185 - val_accuracy: 0.6649 - val_loss: 0.1187
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6826 - loss: 0.1135 - val_accuracy: 0.7087 - val_loss: 0.1039
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7011 - loss: 0.1102 - val_accuracy: 0.7133 - val_loss: 0.1074
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6844 - loss: 0.1171 - val_accuracy: 0.7221 - val

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Fold 2: Using file /kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/train/5-folds/data_part_2.csv as test set
Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.6015 - loss: 0.1455 - val_accuracy: 0.6641 - val_loss: 0.1300
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6283 - loss: 0.1281 - val_accuracy: 0.6344 - val_loss: 0.1198
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6570 - loss: 0.1244 - val_accuracy: 0.6237 - val_loss: 0.1185
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6554 - loss: 0.1199 - val_accuracy: 0.6870 - val_loss: 0.1136
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.6596 - loss: 0.1221 - val_accuracy: 0.6630 - val_loss: 0.1127
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.6619 - loss: 0.1172 - val_accuracy: 0.6775 - val_loss: 0.1218
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6577 - loss: 0.1199 - val_accuracy: 0.713

...

328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.6125 - loss: 0.1576 - val_accuracy: 0.6697 - val_loss: 0.1188
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.6782 - loss: 0.1191 - val_accuracy: 0.7876 - val_loss: 0.0951
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7740 - loss: 0.0926 - val_accuracy: 0.8265 - val_loss: 0.0797
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8172 - loss: 0.0798 - val_accuracy: 0.8375 - val_loss: 0.0577
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8292 - loss: 0.0714 - val_accuracy: 0.8635 - val_loss: 0.0548
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8456 - loss: 0.0648 - val_accuracy: 0.8627 - val_loss: 0.0497
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8473 - loss: 0.0618 - val_accuracy: 0.8631 - val_loss: 0.0480
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8548 - loss: 0.0592 - val_accuracy: 0.8730 - va

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.6026 - loss: 0.1614 - val_accuracy: 0.6892 - val_loss: 0.1156
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.6739 - loss: 0.1187 - val_accuracy: 0.8097 - val_loss: 0.0860
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7759 - loss: 0.0924 - val_accuracy: 0.7879 - val_loss: 0.0983
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8073 - loss: 0.0805 - val_accuracy: 0.7952 - val_loss: 0.0876
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8263 - loss: 0.0725 - val_accuracy: 0.8688 - val_loss: 0.0514
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8478 - loss: 0.0634 - val_accuracy: 0.8741 - val_loss: 0.0455
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8505 - loss: 0.0613 - val_accuracy: 0.8787 - val_loss: 0.0449
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8493 - loss: 0.0627 - val_accuracy: 0.8764 - val

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.6031 - loss: 0.1584 - val_accuracy: 0.6953 - val_loss: 0.1157
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.6885 - loss: 0.1173 - val_accuracy: 0.7742 - val_loss: 0.0929
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.7643 - loss: 0.0967 - val_accuracy: 0.8150 - val_loss: 0.0742
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8068 - loss: 0.0802 - val_accuracy: 0.8665 - val_loss: 0.0526
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8238 - loss: 0.0720 - val_accuracy: 0.7822 - val_loss: 0.0946
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8307 - loss: 0.0720 - val_accuracy: 0.8490 - val_loss: 0.0653
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8400 - loss: 0.0650 - val_accuracy: 0.8650 - val_loss: 0.0472
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8523 - loss: 0.0585 - val_accuracy: 

In [10]:
# Duyệt qua các tuần trong overall_results
for week_result in overall_results_5folds:
    week = week_result["week"]
    average_train_time = np.mean(week_result["train_times"])
    average_test_time = np.mean(week_result["test_times"])
    average_metrics_df = week_result["average_metrics_df"]
    average_confusion_matrix = week_result["average_confusion_matrix"]
    
    # In kết quả
    print(f"\n=== Results for {week} ===")
    print(f"Average Train Time: {average_train_time:.4f} seconds")
    print(f"Average Test Time: {average_test_time:.4f} seconds")
    print("\nAverage Precision, Recall, F1-Score per Label:")
    print(average_metrics_df)
    print("\nAverage Confusion Matrix:")
    print(average_confusion_matrix)



=== Results for week1 ===
Average Train Time: 143.9693 seconds
Average Test Time: 0.8138 seconds

Average Precision, Recall, F1-Score per Label:
   Label  Average Precision  Average Recall  Average F1-Score
0      0           0.656505        0.620000          0.629934
1      1           0.028571        0.004598          0.007921
2      2           0.655446        0.462254          0.537267
3      3           0.559570        0.331737          0.395592
4      4           0.822023        0.933399          0.873315

Average Confusion Matrix:
       0    1     2     3       4
0  372.0  1.4  14.2  22.6   189.8
1   43.0  0.4   2.6   4.8    36.8
2   35.6  1.0  76.0   7.2    44.6
3   51.0  0.0   3.2  55.4    57.4
4   78.6  0.0  21.0   7.2  1496.6

=== Results for week2 ===
Average Train Time: 148.7481 seconds
Average Test Time: 0.8050 seconds

Average Precision, Recall, F1-Score per Label:
   Label  Average Precision  Average Recall  Average F1-Score
0      0           0.865613        0.882333

## Kiểm tra trên tập test

In [11]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

def build_lstm_model(params):
    model = tf.keras.Sequential()
    
    # LSTM layer 1
    model.add(layers.LSTM(
        units=params.get('units_1', 32),
        return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]))
    )
    model.add(layers.Dropout(rate=params.get('dropout_1', 0.2)))

    # LSTM layer 2
    model.add(layers.LSTM(
        units=params.get('units_2', 32),
        return_sequences=True
    ))
    model.add(layers.Dropout(rate=params.get('dropout_2', 0.2)))

    # LSTM layer 3
    model.add(layers.LSTM(
        units=params.get('units_3', 32),
        return_sequences=True
    ))
    model.add(layers.Dropout(rate=params.get('dropout_3', 0.2)))

    # LSTM layer 4
    model.add(layers.LSTM(
        units=params.get('units_4', 32),
        return_sequences=False
    ))
    model.add(layers.Dropout(rate=params.get('dropout_4', 0.2)))

    # Output layer
    model.add(layers.Dense(5, activation='softmax'))  # 5 classes in output

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
        loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
        metrics=['accuracy']
    )
    
    return model

In [12]:
# Mảng lưu dữ liệu của các tuần
results = []

def process_week(week_num, best_params, results):
    print(f"\n=== Processing Week {week_num} ===")
    params = best_params[week].values
    # Đường dẫn tới dữ liệu tuần tương ứng
    train_path = f"/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week{week_num}/train/clean_data_week{week_num}.csv"
    test_path = f"/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week{week_num}/test/test_week{week_num}.csv"
    
    # Load dữ liệu
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    
    # Tách X và y
    X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                       "course_id", "school", "enroll_time", "classification"])
    y_train = train_data['classification_encoded']
    
    X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                     "course_id", "school", "enroll_time", "classification"])
    y_test = test_data['classification_encoded']

    # Áp dụng SMOTE cho tập huấn luyện
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Chuyển đổi nhãn sang dạng one-hot
    y_train_resampled = to_categorical(y_train_resampled, num_classes=5)
    y_test = to_categorical(y_test, num_classes=5)

    # Reshape dữ liệu cho LSTM
    X_train_resampled = X_train_resampled.to_numpy().reshape((X_train_resampled.shape[0], 1, X_train_resampled.shape[1]))
    X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

    # Xây dựng mô hình với tham số tốt nhất
    model = build_lstm_model(params)
    
    # Huấn luyện mô hình
    start_train = time.time()
    model.fit(X_train_resampled, y_train_resampled, epochs=50, validation_split=0.1, batch_size=32)
    end_train = time.time()
    
    # Kiểm thử mô hình
    start_test = time.time()
    y_pred = model.predict(X_test)
    end_test = time.time()
    
    # Tính thời gian huấn luyện và kiểm thử
    train_time = end_train - start_train
    test_time = end_test - start_test
    
    # Đánh giá mô hình
    y_pred_classes = y_pred.argmax(axis=1)
    y_test_classes = y_test.argmax(axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
    
    # Lưu kết quả vào mảng
    results.append({
        "week": week_num,
        "train_time": train_time,
        "test_time": test_time,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": conf_matrix
    })
    
    print("\n=== Precision, Recall, F1-Score per Label ===")
    print(pd.DataFrame({
        "Label": np.unique(y_test_classes),
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    }))
    print("\n=== Confusion Matrix ===")
    print(pd.DataFrame(conf_matrix, index=np.unique(y_test_classes), columns=np.unique(y_test_classes)))
    
    print(f"\nTrain Time: {train_time:.2f} seconds")
    print(f"Test Time: {test_time:.2f} seconds")

In [13]:
process_week(1, best_params, results)


=== Processing Week 1 ===
Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.4379 - loss: 0.2039 - val_accuracy: 0.7067 - val_loss: 0.2430
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5631 - loss: 0.1683 - val_accuracy: 0.7037 - val_loss: 0.2135
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.5812 - loss: 0.1629 - val_accuracy: 0.6840 - val_loss: 0.2170
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.5937 - loss: 0.1597 - val_accuracy: 0.6907 - val_loss: 0.2021
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.6145 - loss: 0.1535 - val_accuracy: 0.6982 - val_loss: 0.2068
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.6162 - loss: 0.1495 - val_accuracy: 0.6875 - val_loss: 0.1794
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.6229 - loss: 0.1485 - val_accuracy: 0.6802 - val_loss: 0.2088
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6174 - loss: 0.1508 - val_a

In [14]:
process_week(2, best_params, results)


=== Processing Week 2 ===
Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.4451 - loss: 0.2032 - val_accuracy: 0.7336 - val_loss: 0.2387
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.5568 - loss: 0.1659 - val_accuracy: 0.7204 - val_loss: 0.2120
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.5775 - loss: 0.1574 - val_accuracy: 0.7306 - val_loss: 0.1960
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.6111 - loss: 0.1453 - val_accuracy: 0.7590 - val_loss: 0.1405
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.6464 - loss: 0.1289 - val_accuracy: 0.7204 - val_loss: 0.1319
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6657 - loss: 0.1205 - val_accuracy: 0.7523 - val_loss: 0.0972
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.6765 - loss: 0.1116 - val_accuracy: 0.7578 - val_loss: 0.1037
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.7033 - loss: 0.1015 - val_a

In [15]:
process_week(3, best_params, results)


=== Processing Week 3 ===
Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.4648 - loss: 0.1961 - val_accuracy: 0.7244 - val_loss: 0.2156
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5807 - loss: 0.1592 - val_accuracy: 0.7234 - val_loss: 0.2199
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6230 - loss: 0.1409 - val_accuracy: 0.6887 - val_loss: 0.1712
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.6697 - loss: 0.1231 - val_accuracy: 0.7877 - val_loss: 0.1211
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.7175 - loss: 0.1041 - val_accuracy: 0.7693 - val_loss: 0.1214
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.7470 - loss: 0.0870 - val_accuracy: 0.8264 - val_loss: 0.0735
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.7731 - loss: 0.0739 - val_accuracy: 0.8785 - val_loss: 0.0428
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.7919 - loss: 0.0634 - val_accu

In [16]:
process_week(4, best_params, results)


=== Processing Week 4 ===
Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.4637 - loss: 0.1978 - val_accuracy: 0.6837 - val_loss: 0.2222
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6078 - loss: 0.1418 - val_accuracy: 0.7002 - val_loss: 0.1796
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.6870 - loss: 0.1096 - val_accuracy: 0.7366 - val_loss: 0.1308
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.7545 - loss: 0.0838 - val_accuracy: 0.7770 - val_loss: 0.0584
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.7911 - loss: 0.0635 - val_accuracy: 0.7116 - val_loss: 0.1098
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.8160 - loss: 0.0540 - val_accuracy: 0.9780 - val_loss: 0.0163
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.8254 - loss: 0.0472 - val_accuracy: 0.8256 - val_loss: 0.0530
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.8341 - loss: 0.0440 - val_acc

In [17]:
# Hiển thị dữ liệu của các tuần
print("\n=== Summary Results for All Weeks ===")
for result in results:
    print(f"Week {result['week']}:")
    print(f"  Train Time: {result['train_time']:.2f} seconds")
    print(f"  Test Time: {result['test_time']:.2f} seconds")
    print(f"  Precision: {result['precision']}")
    print(f"  Recall: {result['recall']}")
    print(f"  F1-Score: {result['f1_score']}")
    print(f"  Confusion Matrix:\n{result['confusion_matrix']}")


=== Summary Results for All Weeks ===
Week 1:
  Train Time: 486.95 seconds
  Test Time: 0.74 seconds
  Precision: [0.85714286 0.25179856 0.52713178 0.69333333 0.87315634]
  Recall: [0.64       0.64814815 0.66019417 0.4952381  0.88534397]
  F1-Score: [0.73282443 0.3626943  0.5862069  0.57777778 0.87920792]
  Confusion Matrix:
[[240  38  22   2  73]
 [  1  35   4   0  14]
 [  7   9  68   2  17]
 [  2  14  12  52  25]
 [ 30  43  23  19 888]]
Week 2:
  Train Time: 478.74 seconds
  Test Time: 0.73 seconds
  Precision: [0.98554913 0.55952381 0.88888889 0.99056604 0.97653959]
  Recall: [0.90933333 0.87037037 0.69902913 1.         0.99601196]
  F1-Score: [0.94590846 0.68115942 0.7826087  0.99526066 0.98617966]
  Confusion Matrix:
[[341   8   2   1  23]
 [  1  47   6   0   0]
 [  1  29  72   0   1]
 [  0   0   0 105   0]
 [  3   0   1   0 999]]
Week 3:
  Train Time: 478.42 seconds
  Test Time: 0.72 seconds
  Precision: [1.         0.60714286 0.95945946 1.         0.99900299]
  Recall: [0.99733